In [1]:
import sys
import datetime
import pandas as pd
from selenium.webdriver.common.by import By
# Browser related import
# Using Edge for performance, but you can choose other browser of your preference
# You may just change 'edge' or 'microsoft' for your personal browser
from selenium.webdriver import Edge as WD # WebDriver
from webdriver_manager.microsoft import EdgeChromiumDriverManager as DM # DriverManager
import selenium.webdriver.edge as swd # selenium web driver
Service = swd.service.Service
Options = swd.options.Options

In [ ]:
# NOT IMPLEMENTED
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

In [ ]:
# NOT IMPLEMENTED
cubes = ['222', '333', '444', '555', '666', '777']

In [2]:
opt = Options()
# For standarization, the browser language is defined to english
opt.add_experimental_option('prefs', {'intl.accept_languages': 'en,en_US'})

In [3]:
driver = WD(service = Service(DM().install()), options = opt)

In [4]:
# This url take us to the past competitions page with every setting needed
competitions = 'https://www.worldcubeassociation.org/competitions?event_ids%5B%5D=333&event_ids%5B%5D=222&event_ids%5B%5D=444&event_ids%5B%5D=555&event_ids%5B%5D=666&event_ids%5B%5D=777&region=all&search=&year=all+years&state=past&from_date=&to_date=&delegate=&display=list'

In [5]:
driver.get(competitions)

In [6]:
# This competitions web element is a list
# with every competition elements scrapped from the page
competitions_we = driver.find_elements(By.XPATH, '//li [@class = "list-group-item past"]')

In [7]:
# In future versions we'll make a DataFrame with all those infos
# So it is good if we already parse the dates provided
def date_parse (x : str):
    (x1 := x.split(','))
    # If date is 'mmm d, yyyy'
    if not x.find(' - '): return datetime.datetime.strptime(x, '%b %d, %Y').date()
    # If date is 'mmm d, yyyy - mmm d, yyyy'
    elif x.count(',') -1: return datetime.datetime.strptime((x.split(' - ')[0]), '%b %d, %Y').date()
    # If date is 'mmm d - d, yyyy' or 'mmm d - mmm d, yyyy'
    else: return datetime.datetime.strptime((f'{x1[0].split(" - ")[0]},' + x1[1]), '%b %d, %Y').date()

In [8]:
# For getting Country an City/State
# If location is 'Country, City' or 'Country, City, State'
country_parse = lambda x: [(x1 := x.split(', '))[0], x1[-1]] 

In [9]:
competition_catalog = {}

init = datetime.datetime.now()
u = 50/(e := len(competitions_we)) # For progress bar, setting the length unit

for i, competition in enumerate(competitions_we):
    
    element = competition.find_element(By.CLASS_NAME, 'competition-link')  
    competition_catalog[element.text] = {
        'url': element.find_element(By.TAG_NAME, 'a').get_attribute('href'),
        'country': (cp := country_parse( # country parse
            competition.find_element(By.CLASS_NAME, 'location').text))[0],
        'city/state': cp[1],        
        'date': date_parse(competition.find_element(By.CLASS_NAME, 'date').text)}
    
    # Printing progress bar
    sys.stdout.write('\r')
    sys.stdout.write(
        f"[{'='*round(i*u)}{' '*round((e-i)*u)}] {round((i/e)*100)}%\
        Elapsed time: {(datetime.datetime.now() - init)}")
    sys.stdout.flush()
print(f'\nDone in {datetime.datetime.now() - init}!')

[==================================================] 100%        Elapsed time: 0:13:21.505526
Done in 0:13:21.508634!


In [10]:
driver.quit()